In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
dataset_path = 'gdrive/My Drive/Projects/Pytorch/Sentiment_analysis/'

In [ ]:
!pip install pytreebank

  Created wheel for pytreebank: filename=pytreebank-0.2.7-cp36-none-any.whl size=37070 sha256=d86caf9238c99bd3015c4c6cbbe3918b1207f7f62fff80ecb63f82b6833e0f29
  Stored in directory: /root/.cache/pip/wheels/e0/b6/91/e9edcdbf464f623628d5c3aa9de28888c726e270b9a29f2368
Successfully built pytreebank


In [ ]:
import pytreebank

In [ ]:
import sys
import os

out_path = os.path.join(dataset_path,sys.path[0], 'sst_{}.txt')
print(out_path)
dataset = pytreebank.load_sst('./raw_data')
# Store train, dev and test in separate files
for category in ['train', 'test', 'dev']:
    with open(out_path.format(category), 'w') as outfile:
        for item in dataset[category]:
            outfile.write("{}\t{}\n".format(
                item.to_labeled_lines()[0][1],
                item.to_labeled_lines()[0][0] + 1,
            ))
# Print the length of the training set
print(len(dataset['train']))

gdrive/My Drive/Projects/Pytorch/Sentiment_analysis/sst_{}.txt
8544


Train, test and dev data is extracted and stored in separate csv files

In [ ]:
import pandas as pd
# Read train data
for file in ['sst_train','sst_test','sst_dev']:
  print(file)
  df = pd.read_csv(dataset_path+file+'.txt', sep='\t', header=None, names=['text', 'truth'])
  #df['truth'] = df['truth'].str.replace('__label__', '')
  df['truth'] = df['truth'].astype(int).astype('category')
  print(df.shape[0])
  print(df.head())
  df.to_csv(dataset_path + file+'.csv',index = False)

sst_train
8544
                                                text truth
0  The Rock is destined to be the 21st Century 's...     4
1  The gorgeously elaborate continuation of `` Th...     5
2  Singer/composer Bryan Adams contributes a slew...     4
3  You 'd think by now America would have had eno...     3
4               Yet the act is still charming here .     4
sst_test
2210
                                                text truth
0                     Effective but too-tepid biopic     3
1  If you sometimes like to go to the movies to h...     4
2  Emerges as something rare , an issue movie tha...     5
3  The film provides some great insight into the ...     3
4  Offers that rare combination of entertainment ...     5
sst_dev
1101
                                                text truth
0  It 's a lovely film with lovely performances b...     4
1  No one goes unindicted here , which is probabl...     3
2  And if you 're not nearly moved to tears by a ...     4
3             

In [ ]:
import torch
from torchtext import data

SEED = 1234
import pandas as pd
import numpy as np
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext

import nltk

import random
from sklearn.metrics import classification_report

#import pyprind
%matplotlib inline 

In [ ]:
import spacy
spacy_en = spacy.load('en')

In [ ]:
is_cuda = torch.cuda.is_available()
print("Cuda Status on system is {}".format(is_cuda))

Cuda Status on system is True


Tokenizer from NLTK is seen to give the best results

In [ ]:
'''from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
stop_words = stopwords.words('english')
table = str.maketrans(string.punctuation, ' '*len(string.punctuation))'''
nltk.download('punkt')
def tokenizer(text):
  return [tok for tok in nltk.word_tokenize(text)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Text and Label fields are created

In [ ]:
TEXT = data.Field(sequential=True, tokenize=tokenizer, include_lengths=True)
LABEL = data.LabelField(dtype=torch.long, sequential=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
df.shape[0]

1101

In [ ]:
train_data, valid_data, test_data = data.TabularDataset.splits(
    path=dataset_path, train="sst_train.csv", 
    validation="sst_dev.csv", test="sst_test.csv",format="csv", skip_header=True, 
    fields=[('text', TEXT), ('truth', LABEL)]
)

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of valid examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 8544
Number of valid examples: 1101
Number of testing examples: 2210


Vocabulary is built using the Glove 300d embedding

In [ ]:
TEXT.build_vocab(train_data, vectors=torchtext.vocab.Vectors(dataset_path+"glove.840B.300d.txt"), 
                 max_size=20000,unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

100%|█████████▉| 2195280/2196017 [05:40<00:00, 7380.07it/s]

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")


Unique tokens in TEXT vocabulary: 18272
Unique tokens in LABEL vocabulary: 5


3 iterators for ttrain, val and test data is created. They are sorted by length of the sentences. Length of any batch is given by the longest sentence in the batch.

In [ ]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# keep in mind the sort_key option 
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), sort_key=lambda x: len(x.text),
    batch_size=BATCH_SIZE,
    sort_within_batch=True,
    device=device)
LABEL.vocab.freqs

Counter({'1': 1092, '2': 2218, '3': 1624, '4': 2322, '5': 1288})

Model is created

1.   Embedding is used with pad token for the pad elements
2.   A bidirectional LSTM is used on the embedded vectors
3.   **Multiplicative Attention** is used to preserve long term dependencies



In [ ]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, dropout, layers, bidirectional_flag, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim,padding_idx=pad_idx)
        #self.embedding.weight.requires_grad = True
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=layers, bidirectional=bidirectional_flag)
        if bidirectional_flag == True:
          self.fc = nn.Linear(2*hidden_dim, output_dim)
        else:
          self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.num_layers = layers
        self.ui = nn.Linear(2*hidden_dim, 100)
        self.ui1 = nn.Linear(2*hidden_dim, 100)
        self.v = nn.Parameter(torch.randn(100))
        self.query = nn.Parameter(torch.randn(1000))
        self.bidirectional_used = bidirectional_flag
        self.hidden_dim = hidden_dim
        
    def forward(self, x, text_lengths, i_hidden):

        #x = [sent len, batch size]
        non_l = nn.ReLU()
        embedded = self.dropout(self.embedding(x))
        #embedded = [sent len, batch size, emb dim]
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded, i_hidden)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        u_it = self.ui(output)
        u_it1 = self.ui1(self.query)
        u_sum = u_it + u_it1
        #weights = torch.softmax((u_it.matmul(self.uw)), dim=0).unsqueeze(2)
        weights = torch.softmax((u_sum.matmul(self.v)), dim=0).unsqueeze(2)
        weights3D = torch.repeat_interleave(weights, 1000, axis=2)
        #print((weights.matmul(output)).size())
        sent = torch.sum(weights3D*output, dim=0)
        #print(sent.size())
        #output, (hidden, cell_state) = self.rnn(embedded)
        #output, hidden = self.rnn(embedded)
        
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        '''
        if self.bidirectional_used == True:
          layer_list = [-1, -2]
        else:
          layer_list = [-1]
        hidden = torch.cat(tuple([hidden[i, :, :] for i in layer_list]), dim=1)'''
        #hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        #assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        #return self.fc(hidden.squeeze(0))
        
        out = self.fc(sent)
        return out

The set of hyperparameters that gave the best result

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 500
OUTPUT_DIM = 5
DROPOUT = 0.5
LAYERS = 3
BIDIRECTIONAL_FLAG = True
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, DROPOUT, LAYERS, BIDIRECTIONAL_FLAG, PAD_IDX)

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([18272, 300])


In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0120,  0.2075, -0.1258,  ...,  0.1387, -0.3605, -0.0350],
        ...,
        [ 0.0495, -0.2737, -0.2819,  ..., -0.2686,  0.5445,  0.1999],
        [ 0.8430, -0.0559, -0.0837,  ...,  0.9208, -0.2708, -0.4322],
        [ 0.4218,  0.2891,  0.6224,  ..., -0.0994, -0.3216, -0.2066]])

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0120,  0.2075, -0.1258,  ...,  0.1387, -0.3605, -0.0350],
        ...,
        [ 0.0495, -0.2737, -0.2819,  ..., -0.2686,  0.5445,  0.1999],
        [ 0.8430, -0.0559, -0.0837,  ...,  0.9208, -0.2708, -0.4322],
        [ 0.4218,  0.2891,  0.6224,  ..., -0.0994, -0.3216, -0.2066]])


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,911,905 trainable parameters


In [ ]:
def binary_accuracy(preds1, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    preds, ind= torch.max(F.softmax(preds1),1)
    correct = (ind == y).float()
    acc = correct.sum()/float(len(correct))
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    #hidden = model.init_hidden(BATCH_SIZE)
    model.train()
    #bar = pyprind.ProgBar(len(iterator), bar_char='█')
    for batch in iterator:
        #print(batch.batch_size)
        optimizer.zero_grad()
        text, text_lengths = batch.text
        hidden = model.init_hidden(batch.batch_size)
        predictions = model(text, text_lengths, hidden).squeeze(1)
      
        loss = criterion(predictions, batch.truth)
        loss.backward()

        acc = binary_accuracy(predictions, batch.truth)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        #bar.update()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    #hidden = model.init_hidden(BATCH_SIZE)
    model.eval()
    
    with torch.no_grad():
        #bar = pyprind.ProgBar(len(iterator), bar_char='█')
        for batch in iterator:
            text, text_lengths = batch.text
            hidden = model.init_hidden(batch.batch_size)
            predictions = model(text, text_lengths, hidden).squeeze(1)

            #predictions = predictions.squeeze(0)
            
            loss = criterion(predictions, batch.truth)
            
            acc = binary_accuracy(predictions, batch.truth)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            #bar.update()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Best model is saved based on validation loss

In [ ]:
N_EPOCHS = 10
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), dataset_path+'model-1.pt')
    #test_loss, test_acc = evaluate(model, test_iterator, criterion)

    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """


| Epoch: 01 | Train Loss: 1.449 | Train Acc: 35.59% | Val. Loss: 1.366 | Val. Acc: 39.71
| Epoch: 02 | Train Loss: 1.340 | Train Acc: 40.91% | Val. Loss: 1.343 | Val. Acc: 39.90
| Epoch: 03 | Train Loss: 1.288 | Train Acc: 43.73% | Val. Loss: 1.309 | Val. Acc: 41.90
| Epoch: 04 | Train Loss: 1.261 | Train Acc: 45.20% | Val. Loss: 1.287 | Val. Acc: 43.47
| Epoch: 05 | Train Loss: 1.222 | Train Acc: 46.86% | Val. Loss: 1.283 | Val. Acc: 44.62
| Epoch: 06 | Train Loss: 1.193 | Train Acc: 48.14% | Val. Loss: 1.294 | Val. Acc: 43.38
| Epoch: 07 | Train Loss: 1.181 | Train Acc: 48.76% | Val. Loss: 1.270 | Val. Acc: 43.70
| Epoch: 08 | Train Loss: 1.143 | Train Acc: 50.80% | Val. Loss: 1.283 | Val. Acc: 43.65
| Epoch: 09 | Train Loss: 1.120 | Train Acc: 52.38% | Val. Loss: 1.279 | Val. Acc: 43.07
| Epoch: 10 | Train Loss: 1.108 | Train Acc: 52.66% | Val. Loss: 1.297 | Val. Acc: 44.36


Test accuracy is seen to be **47.68%**

In [ ]:
model.load_state_dict(torch.load(dataset_path+'model-1.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """


Test Loss: 1.197 | Test Acc: 47.68%
